In [1]:
import pandas as pd
from google.cloud import bigquery # SQL table interface on Arcus

# Initialize the client service
client = bigquery.Client()

# Set the table names
lookup_table = "lab.proc_ord_projects"
project_table = "lab.nf1_scit_walkthrough"

In [2]:
# Get the nf1 table from Ayan's query
q_fn = "../queries/nf1.txt"

with open(q_fn, 'r') as f:
    q = f.read()

df_nf1 = client.query(q).to_dataframe()

print(df_nf1.shape)

(7980, 10)


In [3]:
df_nf1.head(10)

,pat_id,nf1_subject_id,sex,race,dob_year,proc_ord_id,proc_ord_year,start_datetime,proc_ord_age,proc_ord_desc
0,HM405J5F4,CHOP_0750,Male,White,2022,478320465562,2024,2024-07-16 08:03:00,826,MR BRAIN W & W/O IV CONTRAST
1,HM30V1VWB,CHOP_0227,Female,White,2005,478097152933,2024,2024-07-15 18:10:00,6999,MR BRAIN W & W/O IV CONTRAST
2,HM4GPT0G,CHOP_0521,Male,White,2013,478195787244,2024,2024-07-15 07:58:00,4180,MR BRAIN W & W/O IV CONTRAST
3,HM3UJOE8K,CHOP_0725,Male,White,2021,478092803657,2024,2024-07-15 07:39:00,1207,MR BRAIN W & W/O IV CONTRAST
4,HM1ZWI53Y,CHOP_0169,Male,White,1999,478081399250,2024,2024-07-13 09:53:00,9138,MR BRAIN W & W/O IV CONTRAST
5,HM3OEHXQJ,CHOP_0714,Male,White,2019,477663620401,2024,2024-07-12 06:53:00,1674,MR BRAIN W & W/O IV CONTRAST
6,HM2BAHMIN,CHOP_0504,Female,White,2013,478027241089,2024,2024-07-11 07:51:00,3845,MR BRAIN W & W/O IV CONTRAST
7,HM33XWK7U,CHOP_0453,Female,White,2009,478061882763,2024,2024-07-10 16:36:00,5320,MR BRAIN W & W/O IV CONTRAST
8,HM3VT82ZR,CHOP_0724,Male,White,2021,477704461618,2024,2024-07-10 11:28:00,1225,MR BRAIN W & W/O IV CONTRAST
9,HM1R0HSIE,CHOP_0240,Male,White,1994,477517278100,2024,2024-07-09 10:19:00,11080,MR BRAIN W & W/O IV CONTRAST


In [4]:
insert_q = "INSERT INTO "+lookup_table+" (proc_ord_id, pat_id, project) VALUES "
count = 1
q = insert_q
for idx, row in df_nf1.iterrows(): 
    proc_ord_id = row['proc_ord_id']
    pat_id = row['pat_id']
    alt_pat_id = row['nf1_subject_id']
    project = 'NF1'
    q += '("'+proc_ord_id+'", "'+pat_id+'", "'+project+'"), '
    if count % 1000 == 0:
        q = q[:-2]+";"

        # Add the rows to the table structure
        print(q)
        job = client.query(q)
        job.result()
        q = insert_q
        
    count += 1
    
q = q[:-2]+";"

# Add the rows to the table structure
job = client.query(q)
job.result()

INSERT INTO lab.proc_ord_projects (proc_ord_id, pat_id, project) VALUES ("478320465562", "HM405J5F4", "NF1"), ("478097152933", "HM30V1VWB", "NF1"), ("478195787244", "HM4GPT0G", "NF1"), ("478092803657", "HM3UJOE8K", "NF1"), ("478081399250", "HM1ZWI53Y", "NF1"), ("477663620401", "HM3OEHXQJ", "NF1"), ("478027241089", "HM2BAHMIN", "NF1"), ("478061882763", "HM33XWK7U", "NF1"), ("477704461618", "HM3VT82ZR", "NF1"), ("477517278100", "HM1R0HSIE", "NF1"), ("478213174834", "HMWXHKMV", "NF1"), ("477525660576", "HM2781SDJ", "NF1"), ("477472112328", "HM396PXV4", "NF1"), ("477496460141", "HM1UEB0JN", "NF1"), ("477461336425", "HM2QCAEOA", "NF1"), ("477053867963", "HM3X2SYIM", "NF1"), ("476912811084", "HM3M0TNJ7", "NF1"), ("476710161701", "HM3PRWQAW", "NF1"), ("476748861224", "HM3L11OF9", "NF1"), ("476348961554", "HM3EWYQR0", "NF1"), ("476398407545", "HM22KTL2O", "NF1"), ("475999809411", "HMOERHM6", "NF1"), ("475977573785", "HM3DU4NG7", "NF1"), ("476052887822", "HMXLW6NS", "NF1"), ("475982182229", "HM

In [5]:
# Time to check
q = 'select count(distinct proc_ord_id) from '+lookup_table+' where project = "NF1"'
df = client.query(q).to_dataframe()
print(df)

    f0_
0  7980
